# Streamlit ChatBot App using Lancgchain and OpenAI LLM GPT Models

*  OpenAI LLM Chat Models and OpenAI API Key
*  Langchain open source python framework - chains, agents and tools

Basic Steps for Streamlit App to Run locally
*   Install python
*   Open Terminal or Command Line and check python is installed >> python3 --version
*   Install streamlit >> pip3 install streamlit
*   Add python to PATH env variable >> nano ~/.bash_profile >> export PATH="/Library/Frameworks/Python.framework/Versions/3.12/bin:$PATH"
*   Save the file as .py and run the command >> streamlit run appname.py

In [8]:
# Install dependencies
!pip install streamlit
!pip install langchain
!pip install langchain_community
!pip install openai
!pip install -U duckduckgo-search
!pip install arxiv
!pip install wikipedia
!pip install langchain-openai
#!pip install wolfram-alpha

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.5 MB/s eta 0:00:00


In [13]:
import streamlit as st

from langchain.agents import (
    AgentExecutor, AgentType, initialize_agent, load_tools
)

#from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI

import os
os.environ["OPENAI_API_KEY"] = " " # provide your API key from https://platform.openai.com/api-keys

In [16]:
# This function returns AgentExecutor, which is a chain
def load_agent() -> AgentExecutor:

    # streaming = True will result in better user experience since it means that the text response will be updated as it comes in, rather than once all the text has been completed
    llm = ChatOpenAI(model="gpt-4o",temperature=0, streaming=True)

    # DuckDuckGo: A search engine that focuses on privacy; an added advantage is that it doesn’t require developer signup
    # Wolfram Alpha: An integration that combines natural language understanding with math capabilities, for questions like “What is 2x+5 = -3x + 7?”
    # arXiv: Search in academic pre-print publications; this is useful for research-oriented questions
    # Wikipedia: For any question about entities of significant notoriety
    # TODO: try wolfram-alpha!
    tools = load_tools(
        tool_names=["ddg-search", "arxiv", "wikipedia"], # "wolfram-alpha" - requires a key
        llm=llm
    )

    return initialize_agent(
        tools=tools, llm=llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True, handle_parsing_errors=True
    )

In [15]:
import streamlit as st

from langchain.callbacks import StreamlitCallbackHandler

chain = load_agent()

st_callback = StreamlitCallbackHandler(st.container())

if prompt := st.chat_input():
    st.chat_message("user").write(prompt)
    with st.chat_message("assistant"):
        st_callback = StreamlitCallbackHandler(st.container())
        response = chain.run(prompt, callbacks=[st_callback])
        st.write(response)